In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Import and review data

In [2]:
df1 = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv', delimiter="\t")
df1.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
df1 = df1.drop(['id'], axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(1)
memory usage: 390.7+ KB


In [4]:
df2 = pd.read_csv('../input/imdb-review-dataset/imdb_master.csv',encoding="latin-1")
df2.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [5]:
df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
df2.columns = ["review","sentiment"]
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [6]:
df2 = df2[df2.sentiment != 'unsup']
df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [7]:
df = pd.concat([df1, df2]).reset_index(drop=True)
df.head()

,review,sentiment
0,With all this stuff going down at the moment w...,1
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,The film starts with a manager (Nicholas Bell)...,0
3,It must be assumed that those who praised this...,0
4,Superbly trashy and wondrously unpretentious 8...,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 2 columns):
review       75000 non-null object
sentiment    75000 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


## Clean data

In [9]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

%time df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

CPU times: user 2min 51s, sys: 664 ms, total: 2min 52s
Wall time: 2min 52s


In [10]:
df.head()

,review,sentiment,Processed_Reviews
0,With all this stuff going down at the moment w...,1,stuff go moment mj ive start listen music watc...
1,"\The Classic War of the Worlds\"" by Timothy Hi...",1,classic war world timothy hines entertain film...
2,The film starts with a manager (Nicholas Bell)...,0,film start manager nicholas bell give welcome ...
3,It must be assumed that those who praised this...,0,must assume praise film greatest film opera ev...
4,Superbly trashy and wondrously unpretentious 8...,1,superbly trashy wondrously unpretentious 80 ex...


In [11]:
# estimate average length of a review for standard input to the model
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

128.51009333333334

## Model

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential

Using TensorFlow backend.


In [13]:
# Create vocabulary of 6000 words
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
%time tokenizer.fit_on_texts(df['Processed_Reviews'])

CPU times: user 10.7 s, sys: 80 ms, total: 10.8 s
Wall time: 10.8 s


In [14]:
%time list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

CPU times: user 8.03 s, sys: 48 ms, total: 8.08 s
Wall time: 8.08 s


In [15]:
maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 128
epochs = 3
%time model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1)

Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 187s 3ms/step - loss: 0.3747 - acc: 0.8241 - val_loss: 0.2517 - val_acc: 0.9033
Epoch 2/3
60000/60000 [==============================] - 185s 3ms/step - loss: 0.2278 - acc: 0.9105 - val_loss: 0.1830 - val_acc: 0.9370
Epoch 3/3
60000/60000 [==============================] - 182s 3ms/step - loss: 0.1780 - acc: 0.9346 - val_loss: 0.1543 - val_acc: 0.9511
CPU times: user 11min 19s, sys: 45.1 s, total: 12min 4s
Wall time: 9min 16s


## Creating a Submission

In [16]:
df_test=pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv",header=0, delimiter="\t", quoting=3)
df_test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [17]:
df_test["review"] = df_test.review.apply(lambda x: clean_text(x))
df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = df_test["sentiment"]

In [18]:
list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [19]:
%time prediction = model.predict(X_te)
result = (prediction > 0.5).astype(int)
result.shape

CPU times: user 53 s, sys: 344 ms, total: 53.4 s
Wall time: 52.7 s


(25000, 1)

In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(result, y_test)

0.9532546230819516

In [22]:
# Copy the results to a pandas dataframe with an "id" column and a "sentiment" column
output = pd.DataFrame(data={"id":df_test["id"], "sentiment":np.squeeze(result)})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [ ]:
# Use pandas to write the comma-separated output file
output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3)